Deep Learning Course - Exercise 2
This is a softmax prototype, with classes: Happy, Sad, Angry
At first we tried using logistic regression with 2 classes: Happy, Sad
This didn't work well enough, adn we got ~0.5 accuracy.
We tried the same method but to predict Male or Female, and this worked much better with ~0.8 accuracy.
Eventually we tried doing softmax with Happy, Sad, Angry, to see if the problem was a weak model for happy and sad.

In [ ]:
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import timeit
import pandas as pd

In [60]:
# Load the filenames from the folder containing the dataset
filenames = []
for file in os.listdir('audio_softmax'):
    filenames.append(file)
print("num of files=",len(filenames),",num of actors=",len(filenames)/16)

num of files= 512 ,num of actors= 32.0


In [61]:
random.shuffle(filenames)

In [62]:
# max, min = 76117, 53310
# start, end = 21000/sampling_rate, 47424/sampling_rate
# wave_length = end-start
num_train = int(len(filenames)*0.7)
num_test = len(filenames) - num_train

In [63]:
# for filename in filenames[:num_train]:
#     data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast', duration=end, offset=start)
#     print(len(data))

In [64]:
# df = pd.DataFrame(columns=['feature'])
# for index, y in enumerate(filenames):
#     data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast', duration=end, offset=start)
#     mfccs = np.mean(librosa.feature.mfcc(y=data, n_mfcc=25,), axis=0)
#     data_x_train.append(-(mfccs)/100)
    
# df

In [65]:
data_x_train = []
data_x_test = []

start_time = timeit.default_timer()
for filename in filenames[:num_train]:
    data, sampling_rate = librosa.load("audio_softmax/" + filename, sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
    sampling_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
    data_x_train.append(mfccs)
    
for filename in filenames[num_train:]:
    data, sampling_rate = librosa.load("audio_softmax/" + filename, sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
    sampling_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
    data_x_test.append(mfccs)
    
stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

runtime:  60.116553735436355


In [66]:
data_y_train = []
data_y_test = []

for filename in filenames[:num_train]:
    if filename[7] == '3':
        data_y_train.append([1,0,0])
    elif filename[7] == '4':
        data_y_train.append([0,1,0])
    elif filename[7] == '5':
        data_y_train.append([0,0,1])
        
for filename in filenames[num_train:]:
    if filename[7] == '3':
        data_y_test.append([1,0,0])
    elif filename[7] == '4':
        data_y_test.append([0,1,0])
    elif filename[7] == '5':
        data_y_test.append([0,0,1])   

In [67]:
# takes every value in data_y to an array
# ex: [0, 1, 1, 1, 0] => [[0], [1], [1], [1], [0]]
data_y_train_correct = []
data_y_test_correct = []

for val in data_y_train:
    val_arr = []
    val_arr.append(val)
    data_y_train_correct.append(val_arr)
    
for val in data_y_test:
    val_arr = []
    val_arr.append(val)
    data_y_test_correct.append(val_arr)

In [68]:
data_xy_train = []
for i in range(len(data_x_train)):
    data_xy_train.append( (data_x_train[i], data_y_train[i]) )
    
data_xy_test = []
for i in range(len(data_x_test)):
    data_xy_test.append( (data_x_test[i], data_y_test[i]) )

In [69]:
def getXvalues(data_xy):
    x_values = []
    for data in data_xy:
        x_values.append(data[0])
    return x_values

def getYvalues(data_xy):
    y_values = []
    for data in data_xy:
        y_values.append(data[1])
    return y_values

In [70]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

In [98]:
# 0.5779221
features = len(data_xy_train[0][0])

x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 3])
W = tf.Variable(tf.zeros([features, 3]))
b = tf.Variable(tf.zeros([3]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.000009).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

start_time = timeit.default_timer()

for i in range(10000):
    sess.run(train_step, feed_dict={x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)})
    random.shuffle(data_xy_train)
    
stop_time = timeit.default_timer()
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)}))
print('runtime: ', stop_time - start_time)  

0.5779221
runtime:  38.02514685424558


In [75]:
shlomi = []

data, sampling_rate = librosa.load("shlomi3.wav", sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
sampling_rate = np.array(sampling_rate)
mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
shlomi.append(mfccs)

In [95]:
sess.run(y, feed_dict={x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)})


array([[0.3147713 , 0.15764569, 0.527583  ],
       [0.4152123 , 0.56714356, 0.01764412],
       [0.45700225, 0.33541164, 0.20758608],
       [0.27379835, 0.54773325, 0.17846842],
       [0.30680105, 0.21696903, 0.47622985],
       [0.3207436 , 0.60655165, 0.07270476],
       [0.4453827 , 0.47800416, 0.07661314],
       [0.31847003, 0.3898016 , 0.29172838],
       [0.34610218, 0.27572176, 0.3781761 ],
       [0.2863218 , 0.13045374, 0.5832244 ],
       [0.32375515, 0.31821406, 0.35803083],
       [0.28218618, 0.3686446 , 0.3491692 ],
       [0.3681503 , 0.5149641 , 0.11688554],
       [0.31944197, 0.537428  , 0.14312994],
       [0.27557552, 0.21177529, 0.5126492 ],
       [0.47442755, 0.3787719 , 0.14680055],
       [0.39243487, 0.46762434, 0.13994086],
       [0.21975273, 0.668786  , 0.11146124],
       [0.3433473 , 0.41411188, 0.24254085],
       [0.22721922, 0.75307775, 0.01970299],
       [0.3939486 , 0.5674145 , 0.03863689],
       [0.5511571 , 0.337983  , 0.1108598 ],
       [0.

In [58]:
np.shape(shlomi)

(1, 216)